Framework based interaction

In [1]:
import sys
from pathlib import Path
sys.path.append(str(Path.cwd().parent.parent.absolute()))
import config

In [ ]:
from context import Context

context = Context()

In [4]:
from contracts.lk_wrap_contract import LkWrapContract
from contracts.simple_lock_energy_contract import SimpleLockEnergyContract

contract: LkWrapContract = context.get_contracts(config.LK_WRAPS)[0]
energy_contract: SimpleLockEnergyContract = context.get_contracts(config.SIMPLE_LOCKS_ENERGY)[0]
locked_token = energy_contract.locked_token

In [ ]:
contract.set_transfer_role_wrapped_token(context.deployer_account, context.network_provider.proxy, ["erd1qqqqqqqqqqqqqpgq429t3qqmt9tsssf8vksdfte00s64uzpe5zvs0pqqzz"])

upgrade contract

In [ ]:
context.deployer_account.sync_nonce(context.network_provider.proxy)
hash = contract.contract_upgrade(context.deployer_account, context.network_provider.proxy, config.ESCROW_BYTECODE_PATH, [])

Low level interaction

In [ ]:
from multiversx_sdk import TokenPayment, ProxyNetworkProvider
from multiversx_sdk.core.transaction_builders import ContractCallBuilder, ESDTNFTTransferBuilder
from multiversx_sdk.core.transaction_builders import DefaultTransactionBuildersConfiguration
from tools.notebooks import env

config = DefaultTransactionBuildersConfiguration(chain_id=env.CHAIN_ID)
proxy = ProxyNetworkProvider(env.PROXY_URL)

In [ ]:
contract_address = env.WRAPPER_CONTRACT
XMEX = env.XMEX
WXMEX = env.WXMEX

In [ ]:
from multiversx_sdk import AccountNonceHolder, UserSigner
from pathlib import Path

user = env.USER1
user2 = env.USER2
owner = env.OWNER
user_nonce_holder = AccountNonceHolder(proxy.get_account(user).nonce)
user_signer = UserSigner.from_pem_file(Path(env.USER1_PEM))
owner_signer = UserSigner.from_pem_file(Path(env.OWNER_PEM))

Wrap tokens

In [ ]:
from multiversx_sdk import TransactionsConverter

user = env.USER1
user_nonce_holder = AccountNonceHolder(proxy.get_account(user).nonce)

transfers = [
    TokenPayment.meta_esdt_from_amount(XMEX, 46, 1000, 18)
]

builder = ContractCallBuilder(
    config,
    contract=contract_address,
    function_name="wrapLockedToken",
    caller=user,
    call_arguments=[],
    gas_limit=30000000,
    esdt_transfers=transfers
)

tx = builder.build()

converter = TransactionsConverter()
print("Transaction:", converter.transaction_to_dictionary(tx))
print("Transaction data:", tx.data)

In [ ]:
user = env.USER1
user_nonce_holder = AccountNonceHolder(proxy.get_account(user).nonce)

transfers = [
    TokenPayment.meta_esdt_from_amount(env.LKMEX, 4580728, 1000, 18)
]

builder = ContractCallBuilder(
    config,
    contract=contract_address,
    function_name="wrapLockedToken",
    caller=user,
    call_arguments=[],
    gas_limit=30000000,
    esdt_transfers=transfers
)

tx = builder.build()
print("Transaction:", converter.transaction_to_dictionary(tx))
print("Transaction data:", tx.data)

In [ ]:
from multiversx_sdk import TransactionComputer

tx.nonce = user_nonce_holder.get_nonce_then_increment()

tx_computer = TransactionComputer()
tx.signature = user_signer.sign(tx_computer.compute_bytes_for_signing(tx))
proxy.send_transaction(tx)

Unwrap tokens

In [ ]:
user = env.USER1
user_nonce_holder = AccountNonceHolder(proxy.get_account(user).nonce)

transfers = [
    TokenPayment.meta_esdt_from_amount(WXMEX, 1, 1000, 18)
]

builder = ContractCallBuilder(
    config,
    contract=contract_address,
    function_name="unwrapLockedToken",
    caller=user,
    call_arguments=[],
    gas_limit=30000000,
    esdt_transfers=transfers
)

tx = builder.build()
print("Transaction:", converter.transaction_to_dictionary(tx))
print("Transaction data:", tx.data.decode())

In [ ]:
tx.nonce = user_nonce_holder.get_nonce_then_increment()
tx.signature = user_signer.sign(tx_computer.compute_bytes_for_signing(tx))
proxy.send_transaction(tx)

Token transfer

In [ ]:
transfer = TokenPayment.meta_esdt_from_amount(WXMEX, 2, 2000000, 18)

user = env.USER1
receiver = env.USER2
user_nonce_holder = AccountNonceHolder(proxy.get_account(user).nonce)

builder = ESDTNFTTransferBuilder(
    config,
    destination=receiver,
    sender=user,
    gas_limit=30000000,
    payment=transfer
)

tx = builder.build()
print("Transaction:", converter.transaction_to_dictionary(tx))
print("Transaction data:", tx.data)

In [ ]:
tx.nonce = user_nonce_holder.get_nonce_then_increment()
tx.signature = user_signer.sign(tx_computer.compute_bytes_for_signing(tx))
proxy.send_transaction(tx)